---
title: Create Schemas
authors:
  - name: Dimitri Yatsenko
  - date: 2025-01-12
---

# Create Schemas

## What is a schema?

The term schema has two related meanings in the context of databases:

### 1. Schema as a Data Blueprint
A **schema** is a formal specification of the structure of data and the rules governing its integrity.
It serves as a blueprint that defines how data is organized, stored, and accessed within a database.
This ensures that the database reflects the rules and requirements of the underlying business or research project it supports.

In structured data models, such as the relational model, a schema provides a robust framework for defining:
* The structure of tables (relations) and their attributes (columns).
* Rules and constraints that ensure data consistency, accuracy, and reliability.
* Relationships between tables, such as primary keys (unique identifiers for records) and foreign keys (references to related records in other tables).

#### Aims of Good Schema Design
* **Data Integrity**: Ensures consistency and prevents anomalies.
* **Query Efficiency**: Facilitates fast and accurate data retrieval, supports complex queries, and optimizes database performance.
* **Scalability**: Allows the database to grow and adapt as data volumes increase.

#### Key Elements of Schema Design
* **Tables and Attributes**: Each table is defined with specific attributes (columns), each assigned a data type.
* **Primary Keys**: Uniquely identify each record in a table.
* **Foreign Keys**: Establish relationships between entities in tables.
* **Indexes**: Support efficient queries.

Through careful schema design, database architects create systems that are both efficient and flexible, meeting the current and future needs of an organization. The schema acts as a living document that guides the structure, operations, and integrity of the database.

### 2. Schema as a Database Module

In complex database designs, the term "schema" is also used to describe a distinct module of a larger database with its own namespace that groups related tables together. 
This modular approach:
* Separates tables into logical groups for better organization.
* Avoids naming conflicts in large databases with multiple schemas.



# Declaring a schema
Before you can create tables, you must declare a schema to serve as a namespace for your tables.
Each schema requires a unique name to distinguish it within the database.

Here’s how to declare a schema in DataJoint:

In [ ]:
import datajoint as dj

# Define the schema
schema = dj.Schema('schema_name')

[2024-08-27 04:10:41,167][INFO]: Connecting root@localhost:3306
[2024-08-27 04:10:41,184][INFO]: Connected root@localhost:3306


# Using the `schema` Object

The schema object groups related tables together and helps prevent naming conflicts.

By convention, the object created by `dj.Schema` is named `schema`. Typically, only one schema object is used in any given Python namespace, usually at the level of a Python module.

The schema object serves multiple purposes:
* **Creating Tables**: Used as a *class decorator* (`@schema`) to declare tables within the schema. 
For details, see the next section, [Create Tables](010-table.ipynb)
* **Visualizing the Schema**: Generates diagrams to illustrate relationships between tables.
* **Exporting Data**: Facilitates exporting data for external use or backup.

With this foundation, you are ready to begin declaring tables and building your data pipeline.

# Working with Multi-Schema Databases

Organizing larger databases into multiple smaller schemas (or modules) enhances clarity, modularity, and maintainability. In DataJoint, schemas serve as namespaces that group related tables together, while Python modules provide a corresponding organizational structure for the database code.

## Convention: One Database Schema = One Python Module

DataJoint projects are typically organized with each database schema mapped to a single Python module (`.py` file). This convention:

* Promotes modularity by grouping all tables of a schema within one module.
* Ensures clarity by maintaining a single schema object per module.
* Avoids naming conflicts and simplifies dependency management.

Each module declares its own schema object and defines all associated tables. Downstream schemas explicitly import upstream schemas to manage dependencies.

## Dependency Management and Acyclic Design

In multi-schema databases, dependencies between tables and schemas must form a Directed Acyclic Graph (DAG). Cyclic dependencies are not allowed. This ensures:
* Foreign key constraints maintain logical order without forming loops.
* Python module imports align with the dependency structure of the database.

**Key Principles**:
1. Tables can reference each other within a schema or across schemas using foreign keys.
2. Dependencies should be topologically sorted, ensuring upstream schemas are imported into downstream schemas.

# Advantages of Multi-Schema Design
1. **Modularity**: Each schema focuses on a specific aspect of the pipeline (e.g., acquisition, processing, analysis).
2. **Separation of Concerns**: Clear boundaries between schemas simplify navigation and troubleshooting.
3. **Scalability**: Isolated schemas enable easier updates and scaling as projects grow.
4. **Collaboration**: Teams can work on separate modules independently without conflicts.
5. **Maintainability**: Modular design facilitates version control and debugging.

# Defining Complex Databases with Multiple Schemas in DataJoint

In DataJoint, defining **multiple schemas across separate Python modules** ensures that large, complex projects remain well-organized, modular, and maintainable. Each schema should be defined in a **dedicated Python module** to adhere to best practices. This structure ensures that every module maintains **only one `schema` object**, and **downstream schemas import upstream schemas** to manage dependencies correctly. This approach improves code clarity, enables better version control, and simplifies collaboration across teams.

The database schema and its Python module usually have similar names, although they need not be identical. 

Tables can form foreign key dependencies within modules and but also across modules. 
In DataJoint, Such dependencies must be acyclic within each schema: dependencies cannot form closed cycles, so that the graph of dependences forms a DAG (directed acyclic graph). 
Then also database modules form a directed acyclic graph at a higher level: the python modules should never form cyclic import dependences and their database schemas must be topologically sorted in the same way so that tables cannot make foreign key dependencies into tables that are in downstream schemas.


## Why Use Multiple Schemas in Separate Modules?

Using multiple schemas across separate modules offers the following benefits:

1. **Modularity and Code Organization**: Each module contains only the tables relevant to a specific schema, making the codebase easier to manage and navigate.
2. **Clear Boundaries Between Schemas**: Ensures a separation of concerns, where each schema focuses on a specific aspect of the pipeline (e.g., acquisition, processing, analysis).
3. **Dependency Management**: Downstream schemas explicitly **import upstream schemas** to manage table dependencies and data flow.
4. **Collaboration**: Multiple developers or teams can work on separate modules without conflicts.
5. **Scalability and Maintainability**: Isolating schemas into modules simplifies future updates and troubleshooting.


## How to Structure Modules for Multiple Schemas

Below is an example that demonstrates how to organize multiple schemas in separate Python modules.

# Example Project Structure

Here’s an example of how to organize multiple schemas in a DataJoint project:

```
my_pipeline/
│
├── subject.py      # Defines subject_management schema
├── acquisition.py  # Defines acquisition schema (depends on subject_management)
├── processing.py   # Defines processing schema (depends on acquisition)
└── analysis.py     # Defines analysis schema (depends on processing)
```

## Step-by-Step Example

1. `subject.py`:
 * Defines the `subject_management` schema.
 * Contains the Subject table and related entities.
2. `acquisition.py`:
 * Defines the `acquisition` schema.
 * Depends on subject_management for subject-related data.
3. `processing.py`:
 * Defines the `processing` schema.
 * Depends on `acquisition` for data to process.
4. `analysis.py`:
 * Defines the `analysis` schema.
 * Depends on `processing` for processed data to analyze.

By adhering to these principles, large projects remain modular, scalable, and easy to maintain.


In [1]:
%pycat code/subject.py

import datajoint as dj

# Define the subject management schema
schema = dj.Schema("subject_management")

@schema
class Subject(dj.Manual):
    definition = """
    subject_id : int
    ---
    subject_name : varchar(50)
    species : varchar(50)
    """


# Dropping a Schema

Dropping a schema in DataJoint involves permanently deleting all the tables within that schema and the schema itself from the database. This is a significant action because it removes not only the tables but also all the data stored within those tables. To drop a schema, you typically use the `schema.drop()` method, where schema is the schema object you defined earlier in your code. 

When you execute this command, DataJoint will prompt you to confirm the action, as it is irreversible and will result in the loss of all data associated with the schema. This operation is especially powerful because it cascades through all tables within the schema, removing each one. 

It's crucial to ensure that any data within the schema is either no longer needed or has been adequately backed up before dropping the schema, as this action will permanently remove all traces of the data and the schema’s structure from the database.

In [ ]:
# dropping a schema
schema.drop()